In [5]:
module load Singularity/3.5.3
sos dryrun /home/dmc2245/project/UKBB_GWAS_dev/workflow/VCF_QC_pipeline.ipynb \
    qc:1-2\
    --cwd /mnt/mfs/statgen/UKBiobank/data/exome_files/project_VCF/072721_run     \
    --vcfs /mnt/mfs/statgen/UKBiobank/data/exome_files/project_VCF/ukb23156_c21_b0_v1.vcf.gz     \
    --ref_hg38 /home/dmc2245/software/GRCh38_reference_genome/GRCh38_full_analysis_set_plus_decoy_hla.fa    \
    --DP_snp 10     \
    --DP_indel 10    \
    --GQ 20     \
    --AB_snp 0.15     \
    --AB_indel 0.2 \
    --anno_file /mnt/mfs/statgen/UKBiobank/data/exome_files/project_VCF/dbSPN_hg38/00-All.renamechrs.indels_snps.tab.gz    \
    --anno_header /mnt/mfs/statgen/UKBiobank/data/exome_files/project_VCF/dbSPN_hg38/00-All.renamechrs.indels_snps.header.txt\
    --bins 0,5e-05 \
    --container_lmm /mnt/mfs/statgen/containers/lmm.sif 

INFO: Checking qc_1: Split multiallelic sites and create unique variant annotation
HINT: singularity exec  /mnt/mfs/statgen/containers/lmm.sif /bin/bash /mnt/mfs/hgrcgrid/homes/dmc2245/project/UKBB_GWAS_dev/analysis/cluster_scripts/tmptthf625z/singularity_run_18963.sh
python ~/monitor.py bcftools norm -m-any /mnt/mfs/statgen/UKBiobank/data/exome_files/project_VCF/ukb23156_c21_b0_v1.vcf.gz |\
python ~/monitor.py bcftools norm --check-ref w -f /home/dmc2245/software/GRCh38_reference_genome/GRCh38_full_analysis_set_plus_decoy_hla.fa  -Oz|\
python ~/monitor.py bcftools +fill-tags -- -t all,F_MISSING,'VD=sum(DP)' |\
python ~/monitor.py bcftools annotate -x ID -I +'%CHROM:%POS:%REF:%ALT' |\
python ~/monitor.py bcftools annotate -a /mnt/mfs/statgen/UKBiobank/data/exome_files/project_VCF/dbSPN_hg38/00-All.renamechrs.indels_snps.tab.gz  -h /mnt/mfs/statgen/UKBiobank/data/exome_files/project_VCF/dbSPN_hg38/00-All.renamechrs.indels_snps.header.txt -c CHROM,FROM,TO,RS -Oz > /mnt/mfs/statgen/UKBiob

In [ ]:
bcftools norm -m - -f <reference> -Oz -o <normVCF> <inputVCF>
bcftools view -i 'F_PASS(DP>=10 & GQ>=20 & GT!="mis")> 0.9' -Oz -o <filtered_normVCF> <normVCF>

In [ ]:
exome_file_dir="/Bulk/Exome sequences/Population level exome OQFE variants, pVCF format - final release/"

run_wes_qc='''cp /mnt/project//Bulk/Exome\ sequences/Population\ level\ exome\ OQFE\ variants\, pVCF\ format\ \- \final\ release/ukb23157_c22_b0_v1.vcf.gz . ;\
cp /mnt/project/Bulk/Exome\ sequences/Exome\ OQFE\ CRAM\ files/helper_files/GRCh38_full_analysis_set_plus_decoy_hla.* .;\
bcftools norm -m-any ukb23157_c22_b0_v1.vcf.gz -f GRCh38_full_analysis_set_plus_decoy_hla.fa |\
bcftools annotate -x ID -I +'%CHROM_%POS_%REF_%ALT' -Oz > ukb23157_c22_b0_v1.norm.vcf.gz ;\
bcftools view -i 'F_PASS(DP>=10 & GQ>=20 & GT!= "mis")> 0.9' ukb23157_c22_b0_v1.norm.vcf.gz |\
bcftools filter -i 'GT="hom" | TYPE="snp" & GT="het" & (FORMAT/AD[*:1])/(FORMAT/AD[*:0] + FORMAT/AD[*:1]) >= 0.15 | TYPE="indel" & GT="het" & (FORMAT/AD[*:1])/(FORMAT/AD[*:0] + FORMAT/AD[*:1]) >= 0.2' -Oz -o ukb23157_c22_b0_v1.filtered.norm.vcf.gz''' 


In [ ]:
#!/bin/sh

# Requirements:
# Please refer to readme.md for more information about the requirements

# How to Run:
# Run this script using:
# sh 01-WES-QC-norm-filter.sh
# on the command line

# Inputs
# - /gwas_cohort_textfiles/090321_UKBB_Hearing_aid_f3393_expandedwhite_6436cases_96601ctrl_PC1_2.tsv - not strictly needed, but swiss-army-knife needs at least one input

# Outputs
# - ukb23157_c22_b0_v1.filtered.norm.vcf.gz


# Steps:
# 1. copy pVCF files to instance 1-22
# 2. normalize pVCF files
# 3. filter pVCF files
# 4. delete temporary files


# Variables
# set this to the exome path and data field for your release
exome_file_dir="/Bulk/Exome sequences/Population level exome OQFE variants, pVCF format - final release/"
data_field="ukb23157"
# working dir and reference text file dir
data_file_dir="/data/wes_hg38/"
txt_file_dir="/gwas_cohort_textfiles/"

#cmd to run (use as input with `-icmd="{$run_merge}"`)

run_wes_qc='''cp /mnt/project//Bulk/Exome\ sequences/Population\ level\ exome\ OQFE\ variants\, pVCF\ format\ \- \final\ release/ukb23157_c22_b0_v1.vcf.gz . ;\
cp /mnt/project/Bulk/Exome\ sequences/Exome\ OQFE\ CRAM\ files/helper_files/GRCh38_full_analysis_set_plus_decoy_hla.* .;\
bcftools norm -m-any ukb23157_c22_b0_v1.vcf.gz -f GRCh38_full_analysis_set_plus_decoy_hla.fa -Oz -o ukb23157_c22_b0_v1.norm.vcf.gz ;\
bcftools view -i 'F_PASS(DP>=10 & GQ>=20 & GT!= "mis")> 0.9' ukb23157_c22_b0_v1.norm.vcf.gz |\
bcftools filter -i 'GT="hom" | TYPE="snp" & GT="het" & (FORMAT/AD[*:1])/(FORMAT/AD[*:0] + FORMAT/AD[*:1]) >= 0.15 | TYPE="indel" & GT="het" & (FORMAT/AD[*:1])/(FORMAT/AD[*:0] + FORMAT/AD[*:1]) >= 0.2' -Oz -o ukb23157_c22_b0_v1.filtered.norm.vcf.gz''' 


dx run swiss-army-knife -iin="${txt_file_dir}/090321_UKBB_Hearing_aid_f3393_expandedwhite_6436cases_96601ctrl_PC1_2.tsv" \
   -icmd="${run_wes_qc}" --tag="Step1" --instance-type "mem1_ssd1_v2_x16"\
   --destination="${project}:/data/gt_hg19/" --brief --yes